In [130]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load the dataset
Note: we will need to turn the label values into -1 and 1.

In [131]:
import pandas as pd
train = pd.read_csv("/content/gdrive/My Drive/CS 5350/Homework 3/bank-note/train.csv", names=['variance', 'skewness', 'curtosis', 'entropy', 'label'])
print(train.head())

   variance  skewness  curtosis  entropy  label
0  3.848100  10.15390  -3.85610 -4.22280      0
1  4.004700   0.45937   1.36210  1.61810      0
2 -0.048008  -1.60370   8.47560  0.75558      0
3 -1.266700   2.81830  -2.42600 -1.88620      1
4  2.203400   5.99470   0.53009  0.84998      0


In [132]:
train['label'] = train['label'].apply(lambda x: 1 if x == 1 else -1)
print(train.head())

   variance  skewness  curtosis  entropy  label
0  3.848100  10.15390  -3.85610 -4.22280     -1
1  4.004700   0.45937   1.36210  1.61810     -1
2 -0.048008  -1.60370   8.47560  0.75558     -1
3 -1.266700   2.81830  -2.42600 -1.88620      1
4  2.203400   5.99470   0.53009  0.84998     -1


In [133]:
test = pd.read_csv("/content/gdrive/My Drive/CS 5350/Homework 3/bank-note/test.csv", names=['variance', 'skewness', 'curtosis', 'entropy', 'label'])
print(test.head())

   variance  skewness  curtosis   entropy  label
0   3.83840    6.1851  -2.04390 -0.033204      0
1   2.85210    9.1710  -3.64610 -1.204700      0
2   5.24180   10.5388  -4.11740 -4.279700      0
3  -2.26230   12.1177   0.28846 -7.758100      0
4   0.55298   -3.4619   1.70480  1.100800      1


In [134]:
test['label'] = test['label'].apply(lambda x: 1 if x == 1 else -1)
print(test.head())

   variance  skewness  curtosis   entropy  label
0   3.83840    6.1851  -2.04390 -0.033204     -1
1   2.85210    9.1710  -3.64610 -1.204700     -1
2   5.24180   10.5388  -4.11740 -4.279700     -1
3  -2.26230   12.1177   0.28846 -7.758100     -1
4   0.55298   -3.4619   1.70480  1.100800      1


Our Class:

In [135]:
import numpy as np
from sklearn.utils import shuffle

class Perceptron:
  def standard_perceptron(self, X: np.ndarray, Y: np.ndarray, rate: int=1, epoch: int=1):
    """
    This only works when the output is binary in the form of -1 or 1.
    """
    weights: np.ndarray = np.zeros(len(X[0]))
    copy_X = X[:]
    copy_Y = Y[:]
    for _ in range(epoch):
      copy_X, copy_Y = shuffle(copy_X, copy_Y)
      for row, y in zip(copy_X, copy_Y):
        y_pred = np.sign(np.dot(weights, row)).astype(np.int64)
        if y != y_pred:
          weights = weights + rate * (y * row)
    return weights

  def voted_perceptron(self, X: np.ndarray, Y: np.ndarray, rate: int=1, epoch: int=1):
    """
    This only works when the output is binary in the form of -1 or 1.
    returns weights in the form of a list of tuples (weights_i, c_i)
    """
    weights_array: list[tuple] = []
    weights: np.ndarray = np.zeros(len(X[0]))
    c: int = 0

    copy_X = X[:]
    copy_Y = Y[:]

    for _ in range(epoch):
      copy_X, copy_Y = shuffle(copy_X, copy_Y)
      for row, y in zip(copy_X, copy_Y):
        y_pred = np.sign(np.dot(weights, row)).astype(np.int64)
        if y != y_pred:
          weights_array.append((weights, c))
          weights = weights + rate * (y * row)
          c = 1
        else:
          c += 1
    return weights_array

  def average_perceptron(self, X: np.ndarray, Y: np.ndarray, rate: int=1, epoch: int=1):
    """
    This only works when the output is binary in the form of -1 or 1.
    """
    weights: np.ndarray = np.zeros(len(X[0]))
    a: np.ndarray = np.zeros(len(X[0]))
    copy_X = X[:]
    copy_Y = Y[:]
    for _ in range(epoch):
      copy_X, copy_Y = shuffle(copy_X, copy_Y)
      for row, y in zip(copy_X, copy_Y):
        y_pred = np.sign(np.dot(weights, row)).astype(np.int64)
        if y != y_pred:
          weights = weights + rate * (y * row)
        a += weights
    return a

  @staticmethod
  def voted_perceptron_predict(datapoint: np.ndarray, weights: list):
    predictions: float = 0
    for i in range(len(weights)):
      predictions += np.dot(weights[i][0], datapoint) * weights[i][1]
    return np.sign(predictions).astype(np.int32)

  @staticmethod
  def bias_trick(X: np.ndarray):
    """
    Used to add 1 to the input array
    """
    output: list[np.ndarray] = []
    for i in range(len(X)):
      output.append(np.append(X[i], 1))
    return np.array(output)

Quick way to check for if bias_trick works.

In [136]:
array_ex: np.ndarray = np.array([np.array([1, 3, 3]), np.array([2, 3, 3]), np.array([3, 3, 4]), np.array([4, 3, 1])])
print(Perceptron.bias_trick(array_ex))

[[1 3 3 1]
 [2 3 3 1]
 [3 3 4 1]
 [4 3 1 1]]


We will look at the accuracy of the standard perceptron.

In [137]:
print(Perceptron().standard_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10))

[-58.8350927 -30.82818   -39.7006956  -5.670603   58.       ]


Lets make test predictions.

In [138]:
def accuracy_standard_perceptron(y_test: np.ndarray, x_test: np.ndarray, weights: np.ndarray):
  num_observations: int = len(y_test)
  correct: int = 0
  for i in range(num_observations):
    prediction = np.sign(np.dot(weights, x_test[i]))
    if prediction == y_test[i]:
      correct += 1
  return correct / num_observations

In [139]:
weights = Perceptron().standard_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10)
print("standard weights: ", weights)
print("train accuracy: ", accuracy_standard_perceptron(train['label'].values, Perceptron.bias_trick(train.drop('label', axis=1).values), weights))
print("test accuracy: ", accuracy_standard_perceptron(test['label'].values, Perceptron.bias_trick(test.drop('label', axis=1).values), weights))

standard weights:  [-52.6733   -38.93741  -45.327916  -9.518022  57.      ]
train accuracy:  0.9908256880733946
test accuracy:  0.984


Quick check of if everything is as expected.

In [140]:
weights = Perceptron().voted_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10)
print(len(weights))

236


Lets test predictions.

In [141]:
prediction = Perceptron.voted_perceptron_predict(Perceptron.bias_trick(train.drop('label', axis=1).values)[0], weights)
print(prediction, train['label'].values[0])

-1 -1


Lets test accuracy.

In [142]:
def accuracy_voted_perceptron(y_test: np.ndarray, x_test: np.ndarray, weights: list[tuple]):
  num_observations: int = len(y_test)
  correct: int = 0
  for i in range(num_observations):
    prediction = Perceptron.voted_perceptron_predict(x_test[i], weights)
    if prediction == y_test[i]:
      correct += 1
  return correct / num_observations

We need to print the count of the weights along with the weights for the voted perceptron.

In [143]:
weights = Perceptron().voted_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10)
print("train accuracy: ", accuracy_voted_perceptron(train['label'].values, Perceptron.bias_trick(train.drop('label', axis=1).values), weights))
print("test accuracy: ", accuracy_voted_perceptron(test['label'].values, Perceptron.bias_trick(test.drop('label', axis=1).values), weights))

train accuracy:  0.9885321100917431
test accuracy:  0.986


In [144]:
for weight, count in weights:
  print(count, weight)

0 [0. 0. 0. 0. 0.]
1 [-2.9719  -6.8369   0.2702  -0.71291 -1.     ]
4 [-5.8865  -2.7832  -0.18679 -4.74561  0.     ]
2 [-4.8313  -1.5975  -2.82789 -4.63528  1.     ]
1 [ -3.7912  -10.9962   -3.68787   0.69832   0.     ]
6 [-7.8592  -8.0599  -5.88707  0.19748 -1.     ]
6 [-7.2587   -7.06045  -8.09967   0.294879  0.      ]
2 [-10.8814    -3.06465   -8.45812   -3.609821   1.      ]
12 [-11.65601   -4.94145   -6.05582   -2.477921   2.      ]
2 [-10.20591   -1.33475  -10.11152   -4.074521   3.      ]
37 [-11.95381   -7.15775   -4.24162   -2.862521   4.      ]
9 [-10.16631   -2.37775   -9.37782   -6.098721   5.      ]
4 [-9.42203  -6.15005  -7.76472  -4.523321  6.      ]
5 [-10.08211   -9.37605   -3.95892   -3.339721   7.      ]
14 [-11.53001   -4.49665  -12.30172   -1.231121   6.      ]
21 [-13.74831   -5.75065   -9.30312   -0.867341   7.      ]
2 [-18.30141  -18.33605    6.13858   -2.365641   8.      ]
16 [-19.88131  -13.62845   -1.78002   -0.816941   7.      ]
10 [-19.67915  -11.71025   -

Lets check the average perceptron accuracy.

In [145]:
def accuracy_average_perceptron(y_test: np.ndarray, x_test: np.ndarray, weights: np.ndarray):
  num_observations: int = len(y_test)
  correct: int = 0
  for i in range(num_observations):
    prediction = np.sign(np.dot(weights, x_test[i]))
    if prediction == y_test[i]:
      correct += 1
  return correct / num_observations

In [146]:
weights = Perceptron().average_perceptron(X = Perceptron.bias_trick(train.drop('label', axis=1).values), Y = train['label'].values, epoch=10)
print("average perceptron weights: ", weights)
print("train accuracy: ", accuracy_average_perceptron(train['label'].values, Perceptron.bias_trick(train.drop('label', axis=1).values), weights))
print("test accuracy: ", accuracy_average_perceptron(test['label'].values, Perceptron.bias_trick(test.drop('label', axis=1).values), weights))

average perceptron weights:  [-432008.214314   -268086.78152299 -278974.1836      -74294.680935
  359407.        ]
train accuracy:  0.9850917431192661
test accuracy:  0.986
